In [ ]:
!python --version

Python 3.7.3


In [ ]:
!pip install matching 
import pandas as pd 
import numpy as np 
import random 
import joblib
import string 
from matching import Player  
from matching.games import HospitalResident  #case that there are mentors > mentees

#NEED TO CHANGE THE PARAM NAMES 

You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
#this function will format the csv file appropriately, by renaming columns
# @filename: the name of the csv file that needs to be formatted 
# NOT NEEDED!!!
def FormatFile(filename): 
    df = pd.read_csv(filename) 
    df = df.drop(['Mentor: Profession.1'], axis=1) #what if survey column names are different, how to formalize this 
# rename columns
    df.columns = ['Timestamp','mOs','Mindustry','Bregion','mWhy','Mwhy',
              'gender', 'yearsExp', 'CompSize', 'Profession', 'track',
              'ethnicity', 'Growth', 'Groups']
    return df 

In [ ]:
#mapping the dataframe columns to each other 
# @df: the dataframe that we have modified, and are creating a new column out of 
def map_Mwhy(df): 
    var = pd.DataFrame(df['mOs'])
    var = var.join(df['mWhy'])
    var = var.join(df['Mwhy'])

    # looping through df to match mentor and mentee why to 1 array
    new_col =[]
    for index, row in var.iterrows():
        if(row['mOs'] == 'Mentor'):
            new_col.append(row['Mwhy'])
        else:
            new_col.append(row['mWhy'])

    # creating new column of combined why with new column arry
    var['combined_why'] = new_col
    var['combined_why'].fillna("Enhance CV", inplace = True) # replacing nulls

    # adding profession and track columns
    var = var.join(df['Profession'])
    var = var.join(df['track'])
    return var

In [ ]:
#this function maps the professions to a student track 
# @df: the dataframe that we are modifying to map the 

def map_professions(var): 
    mappedprofs = {'Data and Analytics': 'IT', 'Help Desk':'IT', 'Cyber Security':'IT', 'Quality Assurance':'IT', 'Operations':'Financial Operations', 'Finance':'Financial Operations', 'Anti-Money Laundering':'Financial Operations', 
                'Community Management':'Sales and Customer Support', 'Marketing':'Sales and Customer Support', 'Customer Support':'Sales and Customer Support', 'Educator':'Sales and Customer Support', 'Project Management':'Business Operations', 'Content':'Business Operations', 'Media Production':'Business Operations', 
                'Business Development':'Business Operations', 'Sales':'Sales and Customer Support', 'Human Resources':'Business Operations', 'Executive or Founder':'Business Operations', 'Engineering':'Software Development', 'Visual Design':'Software Development', 'UX/UI Design':'Software Development', 'Product Management':'Software Development'}

    maptracks = []
    for index,row in var.iterrows(): 
      if(row['mOs'] == 'Mentor'):
        maptracks.append(mappedprofs[row['Profession']])
      else:
        maptracks.append(row['track'])
    var['combined_track'] = maptracks
    var['combined_track'].fillna("IT", inplace = True) # replacing nulls
    return var

# var['combined_track'] = maptracks
# var['combined_track'].fillna("IT", inplace = True) # replacing nulls

In [ ]:
#joining the data frame with the combined columns and the modified dataframe 
# @df1: the original dataframe with modified column names 
# @df2: the dataframe with the combined tracks and professions
def join_df(df, var): 
    match_df1 = df[['mOs', 'Bregion', 'gender', 'ethnicity', 'Growth', 'Groups']]
    match_df2 = var[['combined_why', 'combined_track']]
    # match dataframe combined!
    match_df = match_df1.join(match_df2)
    return match_df 

In [ ]:
#this function compiles an array of unique ids for the mentors and mentees 
#  and adds the column of ids to the df 
# @df: the dataframe that we are iterating through to assign user ids 
# import string
# import random
def user_id(match_df): 
    letters = string.ascii_lowercase
    unique_id = [] 
    for i in range(0, match_df.shape[0]): 
        unique_id.append(( ''.join(random.choice(letters) for i in range(10)) ))

    # creating column of unique ids  
    match_df["ID"] = unique_id
    return match_df

In [ ]:
# this function fills the df with 0's to prepare for scoring 
# @df: the dataframe we are iterating through in order to add 0's to it 
def score_df(match_df):
    student_subset_df = match_df[match_df['mOs'] == 'Student']
    mentor_subset_df = match_df[match_df['mOs']== 'Mentor']

    match_sc = pd.DataFrame(np.zeros(shape = (student_subset_df.shape[0],mentor_subset_df.shape[0])))
    match_sc.columns = student_subset_df['ID']
    return match_sc 

In [ ]:
#assigning unique ids to the mentors and mentees 
def assign_id(match_df, match_sc): 
    mentor_subset_df = match_df[match_df['mOs']== 'Mentor']
    mentor_subset_df.reset_index(drop=True, inplace=True)  #only thing is how to gain access to the df here 
    match_sc['mentor_id'] = mentor_subset_df['ID']
    match_sc.set_index('mentor_id', inplace = True)
    return match_sc 

In [ ]:
#this function determines the final score for the mentors and mentees 
def final_score(region, no_group, with_group):
    # if(region == 0):     #if the mentor/mentee have dif regions
    #     return 0
    # else:
        # return (no_group + with_group)
    return (no_group + with_group)

In [ ]:
# method returns count of matching objects in array agnostic of index/positionw
# @student: the student array that we are iterating throguh 
# @mentor: the mentor array that we are iterating through 

def group_equal(student, mentor): 
    count = 0
    for x in student: 
        for y in mentor: 
            if x == y: 
                count += 1
    return count 

In [ ]:
#this function inserts the scores into the dataframe 
# @df: match_df, the data frame we are inserting the scores into 
# @return: returns the df with the final scores inserted 
def insert_score(match_sc, match_df):
    for x in match_sc:
        student = match_df[match_df['ID'] == x]
        for y in match_sc.iterrows():
            mentor = match_df[match_df['ID'] == y[0]]
            # capturing matching values between student and mentor w/o group
            student_wo_group = student.drop(['Groups'], axis =1)
            mentor_wo_group = mentor.drop(['Groups'], axis =1)
            score_wo_group = np.count_nonzero(student_wo_group.values==mentor_wo_group.values)
            # capturing region comapirson score
            score_region_comparison = np.count_nonzero(student['Bregion'].values==mentor['Bregion'].values)
            # splitting group column by comma for student and mentor
            student_group = student['Groups'].values[0].split(', ')
            mentor_group = mentor['Groups'].values[0].split(', ')
            # calling group_equal to compare all matching group tags and returns count
            score_group_comparison = group_equal(student_group, mentor_group)
            # replacing value in match_sc with score of student/mentor comparison
            match_sc[x][y[0]] = final_score(score_region_comparison, score_wo_group, score_group_comparison)
    return match_sc

In [ ]:
#this function will return the top 5 matches of students for each mentor 
#@df: the dataframe with the scoress that we are iterating through 
def top_matches_mentor(match_sc): 
    mentor_pref_dict_5 = {}
    mentor_pref_dict = {}
    for x,y in match_sc.iterrows():
        mentor_pref_dict_5[x] = np.array(y.nlargest().index.values)
        mentor_pref_dict[x] = np.array(y.nlargest(len(y)).index.values)
    return mentor_pref_dict

In [ ]:
#this function will return the top 5 mentor matches for each student 
#@df: the dataframe with the scores for mentor/mentee 
def top_matches_student(match_sc): 
    student_pref_dict_5 = {} #store in dict
    student_pref_dict = {} #store in dict
    for x in match_sc:
        student_pref_dict_5[x] = np.array(match_sc[x].nlargest().index.values)
        student_pref_dict[x] = np.array(match_sc[x].nlargest(len(match_sc[x])).index.values)
    return student_pref_dict 

In [ ]:
# @mentordict: the mentor dictionary with the top matches of students 
# returns a df with the matches 
def hospital_resident(mentor_pref_dict, student_pref_dict): 
    capacities = {mentor: 1 for mentor in mentor_pref_dict}

    # documentation: https://github.com/daffidwilde/matching
    game = HospitalResident.create_from_dictionaries(student_pref_dict, mentor_pref_dict, capacities)
    matches = game.solve()
    
    return matches

In [ ]:
def main():
    response_file = FormatFile("responses.csv")
    # print(response_file)

    var = map_Mwhy(response_file)
    # print(v)
    var = map_professions(var)
    # print(a)
    match_df = join_df(response_file, var)
    # print(b)
    match_df = user_id(match_df) #c is match_df 
    # print(c)
    match_sc = score_df(match_df)
    # print(d)
    match_sc = assign_id(match_df, match_sc) #match_sc
    # print(e)
    match_sc = insert_score(match_sc,match_df) # new mathc_sc
    # print(f)
    mentor_pref_dict = top_matches_mentor(match_sc) #returns mentor_pref_df
    # print(g)
    student_pref_dict = top_matches_student(match_sc) #return student_pref
    # print(h)
    matches = hospital_resident(mentor_pref_dict,student_pref_dict)
    print(matches) #returns the matches

    
    

main()

{amzlxgfodc: [kyqtofzyln], evnzdkicgd: [iaivqljhae], pagfunfbwp: [cfhuxdkfxt], esultjnhsa: [kfjqdxvtnu], zphjhsxbdo: [ixkmoihsft], dcjhaudxfo: [dyxtudjijz], ugxykvzckc: [xtsyscatvr], jorcjjutqt: [vqcxrmwjus], jnxwwbokhl: [vryiqkhehi], fqhmmiozlq: [ebgqkkgaip], earfwoblrg: [fdcyxfxfnk], ehfuwqcuar: [ucmheworsp], qklufsauwc: [ijeetbwabq], lgjfrkcxct: [tigmwvbheu], cnoesfktoa: [euegvdjvav], apgvpjlxqk: [pkbxfrectc], sprchdkike: [bzpkftkfue], ljfrrcdbet: [agipiujfcq], dyrtdybrib: [mxkdbcfazc], jiewfrkita: [pejglowipy], bjvycgvkve: [erbvuxynjg], qivfxtzywu: [wfrsuphaue], kyypdrmpmd: [umknvwotsr], saqomaagne: [uiuoohubmr], cotoalgjcm: [yukodxqowt], oscwspvurx: [cdbirbvnqd]}
